In [2]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
import time
from datetime import datetime

# Checking functions

In [3]:
data = pd.read_csv('../data/helpdesk.csv')
#df =  pd.read_csv('../data/helpdesk.csv', index_col=['CompleteTimestamp'], parse_dates=True)

In [4]:
#data['CompleteTimestamp'] = data['CompleteTimestamp'].apply(parse)

In [5]:
data['CompleteTimestamp'] = pd.to_datetime(data['CompleteTimestamp'])

In [6]:
type(data['CompleteTimestamp'])

pandas.core.series.Series

In [7]:
def calculateDuration(df):
    df['Duration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].shift(1)).fillna(0)
    return df

In [8]:
def calculateCumDuration(df):
    df['CumDuration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'][0]).fillna(0)
    return df

In [9]:
def calculateTimeSinceMidNight(x):
    x = str(x)
    x = time.strptime(x, "%Y-%m-%d %H:%M:%S")
    midnight = datetime.fromtimestamp(time.mktime(x)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(x))-midnight
    return timesincemidnight.seconds

In [10]:
def convert2seconds(x):
    x = int(x.total_seconds())
    return x

In [11]:
data = calculateDuration(data)
data = calculateCumDuration(data)

In [12]:
data['Duration'] = data['Duration'].apply(convert2seconds)
data['CumDuration'] = data['CumDuration'].apply(convert2seconds)

In [13]:
data['Timefrommidnight'] = data['CompleteTimestamp'].apply(calculateTimeSinceMidNight)

In [14]:
data['WeekDay'] = data['CompleteTimestamp'].dt.dayofweek

In [15]:
data.head()

,CaseID,ActivityID,CompleteTimestamp,Duration,CumDuration,Timefrommidnight,WeekDay
0,2,1,2012-04-03 16:55:38,0,0,60938,1
1,2,8,2012-04-03 16:55:53,15,15,60953,1
2,2,6,2012-04-05 17:15:52,173999,174014,62152,3
3,3,1,2010-10-29 18:14:06,-45270106,-45096092,65646,4
4,3,8,2010-11-04 01:16:11,457325,-44638767,4571,3


# Preprocessing data

In [16]:
data = pd.read_csv('../data/helpdesk.csv')
data['CompleteTimestamp'] = pd.to_datetime(data['CompleteTimestamp'], errors='coerce')

In [17]:
data.head()

,CaseID,ActivityID,CompleteTimestamp
0,2,1,2012-04-03 16:55:38
1,2,8,2012-04-03 16:55:53
2,2,6,2012-04-05 17:15:52
3,3,1,2010-10-29 18:14:06
4,3,8,2010-11-04 01:16:11


In [18]:
def calculateDuration(df):
    df['Duration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].shift(1)).fillna(0)
    return df

def calculateCumDuration(df):
    df['CumDuration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].iloc[0]).fillna(0) 
    #change df['CompleteTimestamp'][0] --> df['CompleteTimestamp'].iloc[0]
    return df

def calculateTimeSinceMidNight(x):
    x = str(x)
    x = time.strptime(x, "%Y-%m-%d %H:%M:%S")
    midnight = datetime.fromtimestamp(time.mktime(x)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(x))-midnight
    return timesincemidnight.seconds

def convert2seconds(x):
    x = int(x.total_seconds())
    return x

In [19]:
groupByCase = data.groupby(['CaseID'])

In [20]:
len(groupByCase)

3804

In [21]:
df = pd.DataFrame(columns=['CaseID', 'ActivityID', 'CompleteTimestamp', 'Duration', 'CumDuration', 'TimeSinceMidnight', 'WeekDay'])
#Loop all group and apply above functions
for case, group in groupByCase:
    group = calculateDuration(group)
    group = calculateCumDuration(group)
    group['Duration'] = group['Duration'].apply(convert2seconds)
    group['CumDuration'] = group['CumDuration'].apply(convert2seconds)
    group['TimeSinceMidnight'] = group['CompleteTimestamp'].apply(calculateTimeSinceMidNight)
    group['WeekDay'] = group['CompleteTimestamp'].dt.dayofweek
    
    df = df.append(group)

/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [22]:
df.head(20)

,CaseID,ActivityID,CompleteTimestamp,Duration,CumDuration,TimeSinceMidnight,WeekDay
0,2.0,1.0,2012-04-03 16:55:38,0.0,0.0,60938.0,1.0
1,2.0,8.0,2012-04-03 16:55:53,15.0,15.0,60953.0,1.0
2,2.0,6.0,2012-04-05 17:15:52,173999.0,174014.0,62152.0,3.0
3,3.0,1.0,2010-10-29 18:14:06,0.0,0.0,65646.0,4.0
4,3.0,8.0,2010-11-04 01:16:11,457325.0,457325.0,4571.0,3.0
5,3.0,6.0,2010-11-04 01:21:17,306.0,457631.0,4877.0,3.0
6,4.0,1.0,2010-12-15 23:31:53,0.0,0.0,84713.0,2.0
7,4.0,8.0,2010-12-16 17:01:07,62954.0,62954.0,61267.0,3.0
8,4.0,6.0,2010-12-16 17:08:19,432.0,63386.0,61699.0,3.0
9,5.0,1.0,2012-04-03 21:08:32,0.0,0.0,76112.0,1.0


In [27]:
df.to_csv('../input/cleaned_data.csv', index=False)

In [28]:
cleaned_data = pd.read_csv('../input/cleaned_data.csv')

In [29]:
cleaned_data.head()

,CaseID,ActivityID,CompleteTimestamp,Duration,CumDuration,TimeSinceMidnight,WeekDay
0,2.0,1.0,2012-04-03 16:55:38,0.0,0.0,60938.0,1.0
1,2.0,8.0,2012-04-03 16:55:53,15.0,15.0,60953.0,1.0
2,2.0,6.0,2012-04-05 17:15:52,173999.0,174014.0,62152.0,3.0
3,3.0,1.0,2010-10-29 18:14:06,0.0,0.0,65646.0,4.0
4,3.0,8.0,2010-11-04 01:16:11,457325.0,457325.0,4571.0,3.0
